In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# =========================
# 0. 간단 로딩 로그
# =========================
def loading(msg):
    print(f"[LOADING] {msg}")

# =========================
# 1. Selenium 설정
# =========================
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

loading("브라우저 실행")

# =========================
# 2. 무신사 접속
# =========================
driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=M&tabId=brand")
time.sleep(5)
loading("무신사 접속 완료")

# =========================
# 3. Joyride 툴팁 제거 (최대 2개)
# =========================
def close_joyride_tooltip():
    try:
        close_btns = driver.find_elements(
            By.CSS_SELECTOR,
            'button[aria-label="닫기"], button[data-action="primary"]'
        )

        for btn in close_btns:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.5)
            except:
                continue

        if close_btns:
            loading("Joyride 툴팁 제거 완료")

    except:
        pass

# 👉 페이지 로딩 후 2번 시도 (2개 툴팁 대응)
close_joyride_tooltip()
time.sleep(1)
close_joyride_tooltip()

# =========================
# 4. 브랜드 스크롤 컨테이너
# =========================
scroll_container = wait.until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'div[data-scroll-id="menu-sub-area"]')
    )
)
loading("브랜드 메뉴 로딩 완료")

# =========================
# 5. 패션소품 버튼 클릭
# =========================
fashion_btn = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//div[@data-button-name="패션소품"]'
    ))
)

driver.execute_script("arguments[0].click();", fashion_btn)
loading("패션소품 카테고리 선택")
time.sleep(2)

# =========================
# 6. 브랜드 수집 (전체)
# =========================
brand_data = {}

NO_GROWTH_LIMIT = 6   # ← 핵심
no_growth_count = 0

loading("브랜드 전체 수집 시작")

while True:
    before_cnt = len(brand_data)

    brand_items = driver.find_elements(
        By.CSS_SELECTOR,
        'a[href*="/brand/"]'
    )

    for item in brand_items:
        try:
            brand_url = item.get_attribute("href")

            name_box = item.find_element(
                By.CSS_SELECTOR,
                'div.UIBrandItem__NameWrap-sc-1un5tti-4'
            )

            brand_kr = name_box.find_element(
                By.CSS_SELECTOR,
                'span.UIBrandItem__Name-sc-1un5tti-6'
            ).text.strip()

            try:
                brand_en = name_box.find_element(
                    By.CSS_SELECTOR,
                    'span.UIBrandItem__EnglishName-sc-1un5tti-7'
                ).text.strip()
            except:
                brand_en = None

            if brand_url not in brand_data:
                brand_data[brand_url] = {
                    "brand_kr": brand_kr,
                    "brand_en": brand_en,
                    "brand_url": brand_url
                }

        except:
            continue

    after_cnt = len(brand_data)

    # -------------------------
    # 증가 여부 체크
    # -------------------------
    if after_cnt == before_cnt:
        no_growth_count += 1
        loading(f"신규 없음 ({no_growth_count}/{NO_GROWTH_LIMIT})")
    else:
        no_growth_count = 0
        loading(f"신규 브랜드 수집됨 → 총 {after_cnt}개")

    if no_growth_count >= NO_GROWTH_LIMIT:
        loading("스크롤 끝 도달 (전체 수집 완료)")
        break

    # -------------------------
    # virtual scroll 강제 이동
    # -------------------------
    driver.execute_script(
        """
        arguments[0].scrollTop += 300;
        """,
        scroll_container
    )
    time.sleep(0.8)


# =========================
# 7. 결과 정리
# =========================
df = pd.DataFrame(brand_data.values())

loading(f"수집 완료 (총 {len(df)}개)")
print(df)

df.to_csv(
    "musinsa_fashion_goods_brands_test.csv",
    index=False,
    encoding="utf-8-sig"
)

loading("CSV 저장 완료")

[LOADING] 브라우저 실행
[LOADING] 무신사 접속 완료
[LOADING] Joyride 툴팁 제거 완료
[LOADING] 브랜드 메뉴 로딩 완료
[LOADING] 패션소품 카테고리 선택
[LOADING] 브랜드 수집 시작
[LOADING] 테스트용 5개 수집 완료
[LOADING] 수집 완료 (총 5개)
   brand_kr          brand_en                                      brand_url
0  무신사 스탠다드  MUSINSA STANDARD  https://www.musinsa.com/brand/musinsastandard
1       엠엘비               MLB              https://www.musinsa.com/brand/mlb
2      일오공칠    IL-O-GONG-CHIL             https://www.musinsa.com/brand/1507
3     노스페이스    THE NORTH FACE     https://www.musinsa.com/brand/thenorthface
4     아브라함문      ABRAHAM MOON      https://www.musinsa.com/brand/abrahammoon
[LOADING] CSV 저장 완료


In [ ]:
# 브랜드 정보 크롤링

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from bs4 import BeautifulSoup


# ========================================
# 0. CSV 파일에서 전체 URL 읽기
# ========================================
brands_df = pd.read_csv("무신사_전체_브랜드_tags.csv")

brand_urls = brands_df["URL"].head(20).tolist()

print("총 URL 개수:", len(brand_urls))

# ========================================
# 팝업 제거 함수
# ========================================
def close_popups(driver):
    popup_xpaths = [
        "//button[contains(text(), '확인')]",
        "//button[contains(text(), '오늘 그만보기')]",
        "//button[contains(text(), '닫기')]",
        "//button[contains(text(), 'OK')]"
    ]
    for xp in popup_xpaths:
        try:
            btn = WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.XPATH, xp))
            )
            btn.click()
            time.sleep(0.1)
        except:
            pass



# ========================================
# 브랜드 정보 추출 함수
# ========================================
def extract_brand_info(url):

    start_time = time.time()
    print(f"\n⏳ URL 접속 시작 → {url}")

    # Chrome 실행
    options = Options()
    options.add_argument("--window-size=1400,900")
    driver = webdriver.Chrome(options=options)

    driver.get(url)

    # body가 로딩될 때까지 대기
    WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    close_popups(driver)

    # 브랜드 정보 버튼 클릭
    info_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@data-button-name='브랜드 정보']"))
    )
    info_btn.click()
    time.sleep(0.5)
    close_popups(driver)

    # 브랜드 정보 전체 박스 로드
    info_wrap = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.Information__Wrap-sc-1j6oprg-0")
        )
    )

    # 브랜드명
    brand_name_el = info_wrap.find_element(
        By.CSS_SELECTOR,
        "div.Information-sc-1j6oprg-3 span.text-title_18px_med.font-pretendard"
    )
    brand_name = BeautifulSoup(brand_name_el.get_attribute("outerHTML"), 'html.parser').get_text().strip()

    # 설명
    desc_el = info_wrap.find_element(
        By.CSS_SELECTOR,
        "span.text-body_13px_reg.text-center.text-gray-600.font-pretendard"
    )
    brand_description = BeautifulSoup(desc_el.get_attribute("outerHTML"), 'html.parser').get_text().strip()

    driver.quit()

    elapsed = time.time() - start_time

    print(f"✔ 브랜드명: {brand_name}")
    print(f"⏱ 처리 시간: {elapsed:.2f}초")

    return brand_name, brand_description



# ========================================
# 🔥 전체 URL 처리 시작
# ========================================
results = []

print("\n========== 전체 브랜드 크롤링 시작 ==========\n")

for idx, url in enumerate(brand_urls, 1):
    print(f"\n[{idx}/{len(brand_urls)}] 진행 중...")

    try:
        name, desc = extract_brand_info(url)
        results.append({
            "brand_name": name,
            "brand_description": desc
        })
    except Exception as e:
        print(f"❌ 실패 → {url}")
        print(f"이유: {e}")
        results.append({
            "brand_name": None,
            "brand_description": None
        })

print("\n========== 전체 크롤링 완료 ==========\n")

# DataFrame 저장 (시간은 포함되지 않음)
df = pd.DataFrame(results)

# 자동 출력은 하지 않음
# print(df)

# 저장 원하면 아래 주석 해제
df.to_csv("무신사_브랜드_설명_결과.csv", index=False, encoding="utf-8-sig")